In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
# Load Training Data
train_generator = train_datagen.flow_from_directory(
    'train_tomato',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical')

Found 18347 images belonging to 10 classes.


In [4]:
# Load Validation Data
validation_generator = validation_datagen.flow_from_directory(
    'valid_tomato',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical')

Found 4585 images belonging to 10 classes.


In [5]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential

In [6]:
model = Sequential()

In [7]:
model.add(Conv2D(filters=16,kernel_size=1,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=16,kernel_size=1,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [8]:
model.add(Conv2D(filters=32,kernel_size=1,padding='same',activation='relu'))
model.add(Conv2D(filters=32,kernel_size=1,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [9]:
model.add(Conv2D(filters=64,kernel_size=1,padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=1,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [10]:
model.add(Conv2D(filters=128,kernel_size=4,padding='same',activation='relu'))
model.add(Conv2D(filters=128,kernel_size=4,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [11]:
model.add(Conv2D(filters=256,kernel_size=2,padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=2,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [12]:
model.add(Dropout(0.25))   #To avoid overfitting

In [13]:
model.add(Flatten())

In [14]:
model.add(Dense(units=1000,activation='relu'))

In [15]:
model.add(Dropout(0.4))

In [16]:
#Output Layer
model.add(Dense(units=10,activation='softmax'))

In [17]:
model.compile(optimizer=tf.keras.optimizers.Nadam(
    learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])   #Adagrad, Adamax, Nadam

In [18]:
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      64        
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 16)      272       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 32)        544       
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        1056      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                    

In [19]:
training_history = model.fit(x=train_generator,validation_data=validation_generator,epochs=40) ## Epoch set for max accuracy

Epoch 1/40
574/574 [==============================] - 214s 355ms/step - loss: 1.8504 - accuracy: 0.3079 - val_loss: 1.4337 - val_accuracy: 0.4696
Epoch 2/40
574/574 [==============================] - 233s 405ms/step - loss: 1.2377 - accuracy: 0.5472 - val_loss: 1.0221 - val_accuracy: 0.6257
Epoch 3/40
574/574 [==============================] - 202s 352ms/step - loss: 0.9377 - accuracy: 0.6678 - val_loss: 0.7163 - val_accuracy: 0.7431
Epoch 4/40
574/574 [==============================] - 199s 346ms/step - loss: 0.7075 - accuracy: 0.7498 - val_loss: 0.6221 - val_accuracy: 0.7749
Epoch 5/40
574/574 [==============================] - 155s 270ms/step - loss: 0.5686 - accuracy: 0.7966 - val_loss: 0.4502 - val_accuracy: 0.8434
Epoch 6/40
574/574 [==============================] - 119s 208ms/step - loss: 0.4843 - accuracy: 0.8283 - val_loss: 0.3662 - val_accuracy: 0.8656
Epoch 7/40
574/574 [==============================] - 127s 221ms/step - loss: 0.4316 - accuracy: 0.8456 - val_loss: 0.3386 -

In [21]:
# Model Evaluation
val_loss,val_acc = model.evaluate(validation_generator)

144/144 [==============================] - 7s 49ms/step - loss: 0.1331 - accuracy: 0.9525


In [37]:
class_name = validation_generator.class_indices
class_name

{'Tomato___Bacterial_spot': 0,
 'Tomato___Early_blight': 1,
 'Tomato___Late_blight': 2,
 'Tomato___Leaf_Mold': 3,
 'Tomato___Septoria_leaf_spot': 4,
 'Tomato___Spider_mites Two-spotted_spider_mite': 5,
 'Tomato___Target_Spot': 6,
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7,
 'Tomato___Tomato_mosaic_virus': 8,
 'Tomato___healthy': 9}

In [24]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'valid_tomato',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 4585 files belonging to 10 classes.


In [25]:
y_pred = model.predict(test_set)
predicted_categories = tf.argmax(y_pred, axis=1)

144/144 [==============================] - 2s 15ms/step


In [26]:
true_categories = tf.concat([y for x, y in test_set], axis=0)
Y_true = tf.argmax(true_categories, axis=1)

In [27]:
Y_true

<tf.Tensor: shape=(4585,), dtype=int64, numpy=array([0, 0, 0, ..., 9, 9, 9], dtype=int64)>

In [28]:
predicted_categories

<tf.Tensor: shape=(4585,), dtype=int64, numpy=array([8, 8, 6, ..., 8, 6, 8], dtype=int64)>

In [39]:
model.save("tm_2.h5")